In [4]:
# 라이브러리 및 모듈 import
from pycocotools.coco import COCO
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

import warnings
warnings.filterwarnings(action='ignore')

In [5]:
CFG = {
    'NUM_CLASS':34,
    'EPOCHS':10,
    'ACCUMULATE':4,
    'LR':3e-4,
    'BATCH_SIZE':8,
    'SEED':41
}
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [6]:
class CustomDataset(Dataset):

    def __init__(self, annotation, data_dir, transforms=None):
        super().__init__()
        self.data_dir = data_dir
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        image = np.array(Image.open(os.path.join(self.data_dir, image_info['file_name'])).convert('RGB'))
        image = A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3, p=0.8)(image=image)['image']
        image = image.astype(np.float32) / 255.
        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)
        boxes = np.array([x['bbox'] for x in anns])
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        labels = np.array([x['category_id'] for x in anns])
        labels = torch.as_tensor(labels, dtype=torch.int64)
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)
        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}
        if self.transforms:
            while True:
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    target['labels'] = torch.tensor(sample['labels'])
                    break
        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [7]:
def train_transform():
    return A.Compose([
        A.Resize(512, 512),
        #A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3, p=0.8),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def valid_transform():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [8]:
# loss 추적
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

def collate_fn(batch):
    return tuple(zip(*batch))

In [9]:
# https://github.com/rwightman/efficientdet-pytorch/blob/master/effdet/config/model_config.py
def get_net(checkpoint_path=None):
    config = get_efficientdet_config('tf_efficientdet_d0')
    config.num_classes = CFG['NUM_CLASS']
    config.image_size = (512,512)
    
    config.soft_nms = False
    config.max_det_per_image = 25
    
    net = EfficientDet(config, pretrained_backbone=True)
    net.class_net = HeadNet(config, num_outputs=config.num_classes) 
    
    if checkpoint_path:
        checkpoint = torch.load(checkpoint_path)
        net.load_state_dict(checkpoint['model_state_dict'])
    
    return DetBenchTrain(net)
    
# train function
def train_fn(num_epochs, train_loader, optimizer, scheduler, model, device, clip=35):
    model.train()
    step = 0
    for epoch in range(num_epochs):
        with tqdm(train_loader, unit = 'batch') as tepoch:
            for images, targets, _ in tepoch:
                tepoch.set_description(f'epoch {epoch+1}/{num_epochs}')

                images = torch.stack(images) # bs, ch, w, h - 16, 3, 512, 512
                images = images.to(device).float()
                boxes = [target['boxes'].to(device).float() for target in targets]
                labels = [target['labels'].to(device).float() for target in targets]
                target = {"bbox": boxes, "cls": labels}

                # calculate loss
                loss, cls_loss, box_loss = model(images, target).values()
                
                # backward
                (loss / CFG['ACCUMULATE']).backward()
                
                step += 1
                if step % CFG['ACCUMULATE'] : 
                    continue
                # grad clip
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
                optimizer.step()
                optimizer.zero_grad()
                scheduler.step()
                tepoch.set_postfix({'LR':round(scheduler.get_lr()[0],6),'loss':float(loss.detach().cpu()), 'loss_bbox':float(box_loss.detach().cpu()), 'loss_cls':float(cls_loss.detach().cpu())})
            
            torch.save(model.state_dict(), f'./ckp/epoch_{epoch+1}.pth')

In [10]:
annotation = './dataset/train.json'
data_dir = './dataset/train'
train_dataset = CustomDataset(annotation, data_dir, train_transform())
train_data_loader = DataLoader(
    train_dataset,
    batch_size=CFG['BATCH_SIZE'],
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

model = get_net()
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=CFG['LR'])
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, threshold_mode='abs', min_lr=1e-6, verbose=True)
scheduler = CosineAnnealingWarmRestarts(optimizer=optimizer, eta_min=1e-6, T_0=405, T_mult=2)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [ ]:
# labels = []
# for images, targets, _ in tqdm(train_data_loader):
#     labels += [target['labels'].tolist() for target in targets]

In [9]:
# import itertools
# output = list(itertools.chain(*labels))
# np.unique(output,return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]),
 array([500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500], dtype=int64))

In [11]:
train_fn(CFG['EPOCHS'], train_data_loader, optimizer, scheduler, model, device)

epoch 2/10:  14%|█▎        | 111/811 [01:23<09:27,  1.23batch/s, LR=0.000119, loss=911, loss_bbox=0.00572, loss_cls=911]        

In [9]:
# import PIL.Image as Image
# import matplotlib.pyplot as plt
# import cv2
# annotation = './dataset/train.json'
# data_dir = './dataset/train'
# coco = COCO(annotation)
# transforms = A.Compose([A.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4, p=0.8)]) #, A.HueSaturationValue(p = 1)

# image_id = coco.getImgIds(imgIds=2)
# image_info = coco.loadImgs(image_id)[0]

# image = Image.open(os.path.join(data_dir, image_info['file_name'])).convert('RGB')
# image = transforms(image = np.array(image))['image']
# plt.imshow(image)